# Tutorial: Named Entity Recognition

In this short tutorial, we show how to use *ferret* to use and evaluate post-hoc approaches in the task of Named Entity Recognition.

We will use `Babelscape/wikineural-multilingual-ner` as model checkpoint.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

from ferret import (
    Benchmark,
    GradientExplainer,
    IntegratedGradientExplainer,
    LIMEExplainer,
    SHAPExplainer,
)

device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)
device

/home/giuseppe/miniconda3/envs/ferret/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda:0'

In [3]:
model_name = "Babelscape/wikineural-multilingual-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name).to(device)

tokenizer_config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [00:00<00:00, 99.6kB/s]
vocab.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 996k/996k [00:00<00:00, 2.41MB/s]
tokenizer.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1.96M/1.96M [00:00<00:00, 3.74MB/s]
special_tokens_map.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 227kB/s]
config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1.19k/1.19k [00:00<00:00, 2.71MB/s]
model.safetensors: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 709M/709M [00:05<00:00, 127MB/s]


In [4]:
ig = IntegratedGradientExplainer(model, tokenizer, multiply_by_inputs=True)
g = GradientExplainer(model, tokenizer, multiply_by_inputs=True)
s = SHAPExplainer(model, tokenizer)
l = LIMEExplainer(model, tokenizer)

No helper provided. Using default 'text-classification' helper.


In [5]:
bench = Benchmark(model, tokenizer, explainers=[ig, g, s, l], task_name="ner")

Overriding helper for explainer <ferret.explainers.gradient.IntegratedGradientExplainer object at 0x7f6c2fc6ba30>
Overriding helper for explainer <ferret.explainers.gradient.GradientExplainer object at 0x7f6c2fc6b760>
Overriding helper for explainer <ferret.explainers.shap.SHAPExplainer object at 0x7f6c2fc6be80>
Overriding helper for explainer <ferret.explainers.lime.LIMEExplainer object at 0x7f6c2fc6bb50>


In [6]:
example = "My name is John and I live in New York"

In [7]:
bench.score(example, return_dict=True)

{0: ('[CLS]',
  {'O': 0.9994915723800659,
   'B-PER': 2.6596975658321753e-05,
   'I-PER': 0.0001727933413349092,
   'B-ORG': 1.2901788068120368e-05,
   'I-ORG': 6.348660826915875e-05,
   'B-LOC': 9.968474842025898e-06,
   'I-LOC': 0.00011104608711320907,
   'B-MISC': 1.6124546164064668e-05,
   'I-MISC': 9.544820932205766e-05}),
 1: ('My',
  {'O': 0.9999566078186035,
   'B-PER': 8.589762728661299e-06,
   'I-PER': 5.902734756091377e-06,
   'B-ORG': 3.837647000182187e-06,
   'I-ORG': 2.641082573973108e-06,
   'B-LOC': 3.4237652926094597e-06,
   'I-LOC': 3.1093154575501103e-06,
   'B-MISC': 7.962845302245114e-06,
   'I-MISC': 7.820964128768537e-06}),
 2: ('name',
  {'O': 0.9999486207962036,
   'B-PER': 2.6577599783195183e-06,
   'I-PER': 8.545670425519347e-06,
   'B-ORG': 1.2938035069964826e-06,
   'I-ORG': 6.106954060669523e-06,
   'B-LOC': 8.132663538162888e-07,
   'I-LOC': 6.332785687845899e-06,
   'B-MISC': 2.279770342283882e-06,
   'I-MISC': 2.332179428776726e-05}),
 3: ('is',
  {'O':

Let's now extract feature attribution scores to understand the probability the model outputs 
for the class `I-LOC` to the token `York`. 

In [9]:
exp = bench.explain(example, target_token="York", target="I-LOC")

Batch: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 82.80it/s]


In [10]:
exp

[Explanation(text=['My name is John and I live in New York'], tokens=['[CLS]', 'My', 'name', 'is', 'John', 'and', 'I', 'live', 'in', 'New', 'York', '[SEP]'], scores=array([ 0.        ,  0.00383799,  0.04170677,  0.03526182, -0.0290277 ,
         0.01943913, -0.03253587,  0.02645964,  0.14818444,  0.34407989,
         0.31946675,  0.        ]), explainer='Integrated Gradient (x Input)', target_pos_idx=6, helper_type='token-classification', target_token_pos_idx=10, target='I-LOC', target_token='York'),
 Explanation(text=['My name is John and I live in New York'], tokens=['[CLS]', 'My', 'name', 'is', 'John', 'and', 'I', 'live', 'in', 'New', 'York', '[SEP]'], scores=array([ 0.01879721, -0.01355127,  0.04893868, -0.07475153,  0.00767763,
        -0.1387781 ,  0.07703502,  0.16541936, -0.18633376, -0.10591038,
        -0.11951543, -0.04329159], dtype=float32), explainer='Gradient (x Input)', target_pos_idx=6, helper_type='token-classification', target_token_pos_idx=10, target='I-LOC', target

In [11]:
bench.show_table(exp)

,My,name,is,John,and,I,live,in,New,York
Integrated Gradient (x Input),0.00,0.04,0.04,-0.03,0.02,-0.03,0.03,0.15,0.34,0.32
Gradient (x Input),-0.01,0.05,-0.07,0.01,-0.14,0.08,0.17,-0.19,-0.11,-0.12
Partition SHAP,0.00,0.00,0.00,-0.00,0.00,-0.06,0.29,0.24,0.34,0.06
LIME,-0.03,0.02,0.05,-0.04,0.03,0.04,0.17,0.11,0.20,0.30


In [12]:
evaluations = bench.evaluate_explanations(exp)

Explanation eval:   0%|                                                                                                            | 0/4 [00:00<?, ?it/s]NER does not support token removal. 'remove_tokens' set to False
NER does not support token removal. 'remove_tokens' set to False


In [13]:
bench.show_evaluation_table(evaluations)

,aopc_compr,aopc_suff,taucorr_loo
Explainer,,,
Integrated Gradient (x Input),0.88,0.15,0.56
Gradient (x Input),0.01,1.00,0.07
Partition SHAP,0.76,0.24,0.73
LIME,0.88,0.14,0.38
